# Import modules and read the data

In [ ]:
import ROOT
ROOT.enableJSVis()
import root_numpy

In [ ]:
import uproot

import numpy
import pandas
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def copy_tree(outname, tree, entries_start, entries_stop):
    f = ROOT.TFile.Open(outname, 'recreate')
    tree_copy = tree.CloneTree(0)
    for i, entry in enumerate(tree):
        if i < entries_start: continue
        tree_copy.Fill()
        if i > entries_stop: break
    tree_copy.AutoSave()
    f.Close()

In [ ]:
%%time

f1 = ROOT.TFile.Open('data/FV0_sens_100000_048cce8/o2sim.root')
t1 = f1.FindObjectAny('o2sim')

# f2 = ROOT.TFile.Open('data/NEW_100000_onlySensitive_190723_2146/o2sim.root')
f2 = ROOT.TFile.Open('data/FV0_full_100000_048cce8/o2sim.root')
t2 = f2.FindObjectAny('o2sim')

In [ ]:
t = t1

# Helper functions

## PDG

In [ ]:
PDG_CODES = {
            0   : ('unknown', '0'), 
            1   : ('nucleus', 'X'),
            22  : ('gamma', '#gamma'),
#             111 : 'pi0',
            211 : ('pi+', '#pi^{+}'),
           -211 : ('pi-', '#pi^{-}'),
            321 : ('K+', 'K^{+}'),
           -321 : ('K-', 'K^{-}'),
            2212: ('p+', 'p^{+}'),
           -2212: ('p-', 'p^{-}'),
#             2112: 'n',
              11: ('e-', 'e^{-}'),
             -11: ('e+', 'e^{+}'),
              13: ('mu-', '#mu^{-}'),
             -13: ('mu+', '#mu^{+}'),
            3222: ('Sigma+', '#Sigma^{+}'),
           -3222: ('Sigma+_bar', '#bar{#Sigma}^{+}'),
            3112: ('Sigma-', '#Sigma^{-}'),
           -3112: ('Sigma-_bar', '#bar{#Sigma}^{-}'),
#             3122: 'Lambda',
#            -3122: 'Lambda_bar',
            3312: ('Xi-', '#Xi^{-}'),
           -3312: ('Xi-_bar', '#bar{#Xi}^{-}'),
            }

def mysign(x):
    return 1 if x >= 0 else -1
    
PDG_CODES_SORTED = sorted(PDG_CODES.keys(), key=lambda x: abs(x)-0.01*mysign(x))


def get_pdg_name(pdg):
    if pdg > 1e8: 
        return get_pdg_name(1)
    
    if pdg in PDG_CODES.keys():
        return PDG_CODES[pdg][0]
    else:
        print 'particle with PDG code = {} not found'.format(pdg)
        return 'x'

    
def get_pdg_name_latex(pdg):
    if pdg > 1e8: 
        return get_pdg_name(1)
    
    if pdg in PDG_CODES.keys():
        return PDG_CODES[pdg][1]
    else:
        print 'particle with PDG code = {} not found'.format(pdg)
        return 'x'


def get_pdg_index(pdg):
    if pdg > 1e8: return get_pdg_index(1)

    if pdg in PDG_CODES.keys():
        return PDG_CODES_SORTED.index(pdg)
    else:
        print 'particle with PDG code = {} not found'.format(pdg)
        return 0
    
# get_pdg_index(-2212)
# get_pdg_name(3312)

## selection functions

In [ ]:
def is_primary(t, i_ev, i_tr):
    t.GetEvent(i_ev)
    n_tracks = t.GetLeaf('FV0Hit.mTrackID').GetLen()
    if i_tr >= n_tracks:
        print("requested trackID ({}) excesses number of tracks in this event ({})".format(i_ev, n_tracks))
        return -1
    mc_track_id = t.GetLeaf('FV0Hit.mTrackID').GetValue(i_tr)
    
    assert abs(mc_track_id - int(mc_track_id)) < 1e-6
    
    mother_id = t.GetLeaf("MCTrack.mMotherTrackId").GetValue(int(mc_track_id))
    if mother_id < 0: return 1
    elif mother_id >= 0: return 0

    
def is_secondary(t, i_ev, i_tr):
    return not is_primary(t, i_ev, i_tr)
    

def is_electron(t, i_ev, i_tr):
    t.GetEvent(i_ev)
    pdg = int(t.GetLeaf('FV0Hit.mParticlePdg').GetValue(i_tr))
    return abs(pdg) == 11

def is_not_electron(t, i_ev, i_tr):
    return not is_electron(t, i_ev, i_tr)

def is_gamma(t, i_ev, i_tr):
    t.GetEvent(i_ev)
    pdg = int(t.GetLeaf('FV0Hit.mParticlePdg').GetValue(i_tr))
    return pdg == 22 

def is_not_gamma(t, i_ev, i_tr):
    return not is_gamma(t, i_ev, i_tr)


def has_passed(t, i_ev, i_tr):
    t.GetEvent(i_ev)
    return t.GetLeaf('FV0Hit.mPos.fCoordinates.fZ').GetValue(i_tr) > 319.9

def has_not_passed(t, i_ev, i_tr):
    return not has_passed(t, i_ev, i_tr)

def has_passed_shift(t, i_ev, i_tr):
    t.GetEvent(i_ev)
    return t.GetLeaf('FV0Hit.mPos.fCoordinates.fZ').GetValue(i_tr) > 319.9+2

def has_not_passed_shift(t, i_ev, i_tr):
    return not has_passed_shift(t, i_ev, i_tr)


def is_in_sector(t, i_ev, i_tr, sectorID):
    val = t.GetLeaf('FV0Hit.mDetectorID').GetValue(i_tr)
    return int(val) == sectorID



is_primary(t, 3, 10)

## histo extraction

In [ ]:
def get_histo(t, var, binning=None, f_apply=lambda x:x, select_funcs=[], events_frac=1e-2, verbose=False):
    if not binning: 
        binning = (10000, t.GetMinimum(var), t.GetMaximum(var))
        
    hname = 'h_'+var.replace('.', '_')
    h = ROOT.TH1D(hname, hname, *binning)
    leaf = t.GetLeaf(var)
    if verbose: print '# tree entries = ', t.GetEntries()
    if verbose: print 'leaf lengths:'
    if verbose: leafs = []
    for i_ev in range(int(t.GetEntries() * events_frac)):
        t.GetEvent(i_ev)
        if verbose:
            if not i_ev % 5000: print i_ev
        if verbose: print leaf.GetLen(),
        if verbose: leafs.append(leaf.GetLen())
        for i_tr in range(leaf.GetLen()):
            val = leaf.GetValue(i_tr)
            if not all([fun(t, i_ev, i_tr) for fun in select_funcs]): 
                continue 
            h.Fill(f_apply(val))
    h.SetTitle(var)
    if verbose: print np.mean(leafs), ' +/- ', np.std(leafs)
    if verbose: print
    if verbose: print '#entries = {}'.format(h.GetEntries())
    return h


def get_histo_2D(t, var1, var2, binning=None, f_apply=[lambda x:x, lambda x:x], select_funcs=[], events_frac=1e-1):
    if not binning: 
        binning = [(100, t.GetMinimum(var2), t.GetMaximum(var2)), (100, t.GetMinimum(var2), t.GetMaximum(var2))] 
        
    hname = 'h_'+var1.replace('.', '_')+'_vs_'+var2.replace('.', '_')
    h = ROOT.TH2D(hname, hname, 
                  binning[0][0], binning[0][1], binning[0][2],
                  binning[1][0], binning[1][1], binning[1][2])
    leaf1 = t.GetLeaf(var1)
    leaf2 = t.GetLeaf(var2)
    for i_ev in range(int(t.GetEntries() * events_frac)):
        t.GetEvent(i_ev)
        if not i_ev % 5000: print i_ev
        for i_tr in range(leaf1.GetLen()):
            val1 = leaf1.GetValue(i_tr)
            val2 = leaf2.GetValue(i_tr)
            if not all([fun(t, i_ev, i_tr) for fun in select_funcs]): 
                continue 
            h.Fill(f_apply[0](val1), f_apply[1](val2))
    h.SetTitle(hname)
    print '#entries = {}'.format(h.GetEntries())
    return h

## gStyle and detectors id mapping

In [ ]:
def detector_id_new2old(new_id):
    n = 1+(new_id-1)*8
    return (n % 40) + (n // 40)

def detector_id_old2new(old_id):
    n = 1+(old_id-1)*5
    return (n % 40) + (n // 40)

# for i in range(1,41):
#     print 'i = {:3d}\tnew2old = {:3d}\told2new = {:3d}\told2new2old = {:3d}\tnew2old2new = {}'.format(i, detector_id_new2old(i), detector_id_old2new(i), detector_id_new2old(detector_id_old2new(i)), detector_id_old2new(detector_id_new2old(i)))

In [ ]:
n_bins = 100
BINNINGS = {
            'mTrackID':(1000, 0, 2000),
            'mPos.fCoordinates.fX':(n_bins, -75, 75),
            'mPos.fCoordinates.fY':(n_bins, -75, 75),
            'mPos.fCoordinates.fZ':(n_bins, 315.8, 320.2),
            'mTime':(n_bins, -1e-6, 1e-5),
            'mHitValue':(n_bins*5, 0, 0.05),
            'mELoss':(n_bins, 0, 0.1),
            'mDetectorID':(43,0,43),
            'mMomentumStart.fCoordinates.fX':(n_bins, -10, 10),
            'mMomentumStart.fCoordinates.fY':(n_bins, -10, 10),
            'mMomentumStart.fCoordinates.fZ':(n_bins,  -20, 500),
            'mPositionStart.fCoordinates.fX':(n_bins*4, -75, 75),
            'mPositionStart.fCoordinates.fY':(n_bins, -75, 75),
            'mPositionStart.fCoordinates.fZ':(n_bins,  315.8, 320.2),
            'mEnergyStart':(n_bins, 0, 200),
            'mParticlePdg':(len(PDG_CODES), 0, len(PDG_CODES))
    }

def root_style_default():
    ROOT.gStyle.SetOptStat(111110)
    ROOT.gStyle.SetStatX(0.9)
    ROOT.gStyle.SetStatW(0.8)
    ROOT.gStyle.SetStatY(0.95)
    ROOT.gStyle.SetStatH(0.15)
    ROOT.gStyle.SetPadTopMargin(0.23)
    ROOT.gStyle.SetTitleY(1.0)
    ROOT.gStyle.SetHistTopMargin(0.8)
    # ROOT.gStyle.SetHistMinimumZero(1)
    
    ROOT.gStyle.SetTitleFontSize(0.05)

    ROOT.gStyle.SetOptLogy(1)

    ROOT.gStyle.SetHistLineWidth(2)
    
root_style_default()

_________________________________________________

# Sample plots

In [ ]:
ROOT.gStyle.SetOptLogy(0)
ROOT.gStyle.SetOptStat(0)

det_id = 18
print detector_id_old2new(det_id),  det_id

# h1 = get_histo_2D(t1, 'FV0Hit.mPositionStart.fCoordinates.fX', 'FV0Hit.mPositionStart.fCoordinates.fY', 
#                  binning=((50,-80,80),(50,-80,80)), 
#                  select_funcs=[lambda x,y,z: is_in_sector(x,y,z, detector_id_old2new(det_id)), ])
h1 = get_histo_2D(t1, 'FV0Hit.mPositionStart.fCoordinates.fX', 'FV0Hit.mPositionStart.fCoordinates.fY', 
                 binning=((50,-80,80),(50,-80,80)), 
                 select_funcs=[lambda x,y,z: is_in_sector(x,y,z, det_id), ])
h2 = get_histo_2D(t2, 'FV0Hit.mPositionStart.fCoordinates.fX', 'FV0Hit.mPositionStart.fCoordinates.fY', 
                 binning=((50,-80,80),(50,-80,80)), 
                 select_funcs=[lambda x,y,z: is_in_sector(x,y,z, det_id), ])

c = ROOT.TCanvas()
c.Divide(2,1)
c.cd(1)
h1.Draw("colz")
c.cd(2)
h2.Draw("colz")
c.Draw()

In [ ]:
old_suffix = 'old'
new_suffix = 'new'

old_color = ROOT.kMagenta+2
new_color = ROOT.kGreen+2

In [ ]:
def get_histo_n_prim_sec(tree, select_funcs, suffix, color):
    h_p = get_histo(tree, 'FV0Hit.mTrackID', binning=BINNINGS['mTrackID'], select_funcs=select_funcs+[is_primary,])
    h_s = get_histo(tree, 'FV0Hit.mTrackID', binning=BINNINGS['mTrackID'], select_funcs=select_funcs+[is_secondary,])
    h_n = ROOT.TH1F('h_n_'+suffix, 'h_n_'+suffix, 2, 0, 2)
    for _ in range(int(h_p.GetEntries())): h_n.Fill(0)
    for _ in range(int(h_s.GetEntries())): h_n.Fill(1)

    h_n.GetXaxis().SetBinLabel(1, 'primary')
    h_n.GetXaxis().SetBinLabel(2, 'secondary')
    h_n.SetLineColor(color)

    return h_n



root_style_default()

ROOT.gStyle.SetHistMinimumZero(1)
ROOT.gStyle.SetOptLogy(0)
ROOT.gStyle.SetHistTopMargin(0.2)
ROOT.gStyle.SetHistLineWidth(3)

h_n_old = get_histo_n_prim_sec(t1, [], suffix=old_suffix, color=old_color)
h_n_new = get_histo_n_prim_sec(t2, [], suffix=new_suffix, color=new_color)

c = ROOT.TCanvas('c','c', 800, 400)
c.Divide(2,1)
c.cd(1)
h_n_old.DrawCopy()
c.cd(2)
h_n_new.DrawCopy()
c.Draw()

In [ ]:
def get_histo_pdg(tree, **kwargs):
    h_pdg = get_histo(tree, 'FV0Hit.mParticlePdg', [len(PDG_CODES), 0, len(PDG_CODES)], get_pdg_index, **kwargs)
    for i, code in enumerate(PDG_CODES_SORTED,1):
        name = get_pdg_name_latex(code)
        h_pdg.GetXaxis().SetBinLabel(i, name)
    h_pdg.SetLabelSize(0.06)

    return h_pdg


    
c = ROOT.TCanvas()
hp = get_histo_pdg(t1, select_funcs=[is_primary])
hs = get_histo_pdg(t1, select_funcs=[is_secondary])
hs.SetLineColor(ROOT.kRed)
hp.DrawNormalized()
hs.DrawNormalized('same')
c.Draw()

In [ ]:
ROOT.gStyle.SetOptLogy(0)
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetPadTopMargin(0.1)

c = ROOT.TCanvas()
h = get_histo_2D(t1, 'FV0Hit.mPositionStart.fCoordinates.fX', 'FV0Hit.mPositionStart.fCoordinates.fY', 
                 select_funcs=[is_primary],
#                  binning=(BINNINGS['mPositionStart.fCoordinates.fX'], BINNINGS['mPositionStart.fCoordinates.fY'])
                 binning=((30, -20, 20), (30, -20, 20))
                )
h.Draw('colz')
c.Draw()

In [ ]:
# %load_ext line_profiler
# %lprun -f get_histo_2D get_histo_2D(t1, 'V0Hit.mPositionStart.fCoordinates.fX', 'V0Hit.mPositionStart.fCoordinates.fY', select_funcs=[is_primary], binning=(BINNINGS['mPositionStart.fCoordinates.fX'], BINNINGS['mPositionStart.fCoordinates.fY']))

In [ ]:
[leaf.GetName() for leaf in t.GetListOfLeaves() if 'V0Hit.' in leaf.GetName()]

In [ ]:
%%time

root_style_default()

c = ROOT.TCanvas('','',1600,1200)
c.Divide(5,3)
for i,var in enumerate([leaf.GetName() for leaf in t.GetListOfLeaves() if 'V0Hit.' in leaf.GetName()],1):
#     if i > 8 or i < 5: continue
    print
    print var
    c.cd(i)
    binning = BINNINGS[var.replace('FV0Hit.', '')]
    is_pdg = 'pdg' in var.lower()
    f_apply = get_pdg_index if is_pdg else lambda x: x
#     h  = get_histo(t, var, f_apply=f_apply, binning=binning, select_funcs=[])
    hp = get_histo(t, var, f_apply=f_apply, binning=binning, select_funcs=[is_primary])
    hs = get_histo(t, var, f_apply=f_apply, binning=binning, select_funcs=[is_secondary])
    h  = ROOT.TH1D()
    hp.Copy(h)
    h.Add(hs)
    if is_pdg:   
        for i, code in enumerate(PDG_CODES_SORTED,1):
            name = get_pdg_name_latex(code)
            h.GetXaxis().SetBinLabel(i, name)
       
    h.SetLineColor(ROOT.kWhite)
    hs.SetLineColor(ROOT.kRed)
    h.Scale(1/h.GetEntries())
    hp.Scale(1/hp.GetEntries())
    hs.Scale(1/hs.GetEntries())

    h.DrawCopy('hist')
    hp.DrawCopy('hist same')
    hs.DrawCopy('hist same')


c.Draw()


_____________________
_____________________
_____________________
_____________________

# FINAL PLOTS - sensOnly vs Full sim.

## sensitiveOnly vs full comparison

In [ ]:
old_suffix = 'sensitive part only'
new_suffix = 'full'
old_suffix_short = 'sensOnly'
new_suffix_short = 'full'
output_dir = 'graphics/tmp'

old_color = ROOT.kMagenta+2
new_color = ROOT.kGreen+2

In [ ]:
[leaf.GetName() for leaf in t.GetListOfLeaves() if 'V0Hit.' in leaf.GetName()]

### number of secondaries

hist, 1 bar for primary and 1 bar for secondary  
no norm

In [ ]:
%%time

root_style_default()

ROOT.gStyle.SetHistMinimumZero(1)
ROOT.gStyle.SetOptLogy(0)
ROOT.gStyle.SetHistTopMargin(0.2)
ROOT.gStyle.SetHistLineWidth(3)


h_n_old = get_histo_n_prim_sec(t1, [], suffix=old_suffix, color=old_color)
h_n_new = get_histo_n_prim_sec(t2, [], suffix=new_suffix, color=new_color)
# h_n_old, h_n_new = plot_n_prim_sec()

c = ROOT.TCanvas('c','c', 800, 400)
c.Divide(2,1)
c.cd(1)
h_n_old.DrawCopy()
c.cd(2)
h_n_new.DrawCopy()
c.Draw()

### all-grid for primaries

expected: little difference

In [ ]:
%%time

root_style_default()

c = ROOT.TCanvas('c','c',2400,1200)
c.Divide(5,3)
for i,var in enumerate([leaf.GetName() for leaf in t.GetListOfLeaves() if 'V0Hit.' in leaf.GetName()],1):
#     if i > 3: break
    print var,

    is_pdg = 'pdg' in var.lower()
    is_det_id = 'detectorid' in var.lower()
    is_trk_id = 'trackid' in var.lower()

    if is_trk_id:
        h_old = get_histo_n_prim_sec(t1, [], suffix=old_suffix, color=old_color)
        h_new = get_histo_n_prim_sec(t2, [], suffix=new_suffix, color=new_color)
    else:
        binning = BINNINGS[var.replace('FV0Hit.', '')]
        if is_pdg:
            f_apply1 = get_pdg_index 
            f_apply2 = get_pdg_index 
        elif is_det_id:
#             f_apply1 = detector_id_new2old
            f_apply1 = lambda x: x
            f_apply2 = lambda x: x
        else:
            f_apply1 = lambda x: x
            f_apply2 = lambda x: x
        h_old = get_histo(t1, var, f_apply=f_apply1, binning=binning, select_funcs=[is_primary])
        h_new = get_histo(t2, var, f_apply=f_apply2, binning=binning, select_funcs=[is_primary])

    h_sum  = ROOT.TH1D()
    h_old.Copy(h_sum)
    h_sum.Add(h_new)
    
    if is_trk_id:
        h_sum.SetTitle('n prim. & seco.')
        h_sum.SetLabelSize(0.07)

    if is_pdg:   
        for i_bin, code in enumerate(PDG_CODES_SORTED,1):
            name = get_pdg_name_latex(code)
            h_sum.GetXaxis().SetBinLabel(i_bin, name)
        h_sum.SetLabelSize(0.07)
    h_sum.SetLineColor(ROOT.kWhite)
    h_old.SetLineColor(old_color)
    h_new.SetLineColor(new_color)
    h_sum.Scale(1/h_sum.GetEntries())
    h_old.Scale(1/h_old.GetEntries())
    h_new.Scale(1/h_new.GetEntries())
    
    
    c.cd(i)
    if is_trk_id:
        ROOT.gPad.SetLogy(0)
#         h_sum.SetMinimum(0)
    else:
        ROOT.gPad.SetLogy(1)
    h_sum.DrawCopy('hist')
    h_old.DrawCopy('hist same')
    h_new.DrawCopy('hist same')
    if is_trk_id: 
        legend = ROOT.TLegend(0.1, 0.63, 0.85, 0.76)
        le_old = legend.AddEntry(h_old, old_suffix, 'l')
        le_new = legend.AddEntry(h_new, new_suffix, 'l')
        le_old.SetLineWidth(6)
        le_new.SetLineWidth(6)
        le_old.SetLineColor(old_color)
        le_new.SetLineColor(new_color)
        legend.Draw()


c.Draw()
c.SaveAs('{}/grid_prim__{}_vs_{}.png'.format(output_dir, old_suffix_short, new_suffix_short))
c.SaveAs('{}/grid_prim__{}_vs_{}.pdf'.format(output_dir, old_suffix_short, new_suffix_short))

### all-grid for secondaries
expected: larger difference, where?

In [ ]:
%%time

root_style_default()

c = ROOT.TCanvas('c','c',2400,1200)
c.Divide(5,3)
for i,var in enumerate([leaf.GetName() for leaf in t.GetListOfLeaves() if 'V0Hit.' in leaf.GetName()],1):
    print var,

    is_pdg = 'pdg' in var.lower()
    is_det_id = 'detectorid' in var.lower()
    is_trk_id = 'trackid' in var.lower()

    if is_trk_id:
        h_old = get_histo_n_prim_sec(t1, [], suffix=old_suffix, color=old_color)
        h_new = get_histo_n_prim_sec(t2, [], suffix=new_suffix, color=new_color)
    else:
        binning = BINNINGS[var.replace('FV0Hit.', '')]
        if is_pdg:
            f_apply1 = get_pdg_index 
            f_apply2 = get_pdg_index 
        elif is_det_id:
#             f_apply1 = detector_id_new2old
            f_apply1 = lambda x: x
            f_apply2 = lambda x: x
        else:
            f_apply1 = lambda x: x
            f_apply2 = lambda x: x
        h_old = get_histo(t1, var, f_apply=f_apply1, binning=binning, select_funcs=[is_secondary])
        h_new = get_histo(t2, var, f_apply=f_apply2, binning=binning, select_funcs=[is_secondary])
    
    h_sum  = ROOT.TH1D()
    h_old.Copy(h_sum)
    h_sum.Add(h_new)
    if is_trk_id:
        h_sum.SetTitle('n prim. & seco.')
        h_sum.SetLabelSize(0.07)

    if is_pdg:    
        for i_bin, code in enumerate(PDG_CODES_SORTED,1):
            name = get_pdg_name_latex(code)
            h_sum.GetXaxis().SetBinLabel(i_bin, name)
        h_sum.SetLabelSize(0.07)

    h_sum.SetLineColor(ROOT.kWhite)
    h_old.SetLineColor(old_color)
    h_new.SetLineColor(new_color)
    h_sum.Scale(1/h_sum.GetEntries())
    h_old.Scale(1/h_old.GetEntries())
    h_new.Scale(1/h_new.GetEntries())
    
    c.cd(i)
    if is_trk_id:
        ROOT.gPad.SetLogy(0)
    else:
        ROOT.gPad.SetLogy(1)
    h_sum.DrawCopy('hist')
    h_old.DrawCopy('hist same')
    h_new.DrawCopy('hist same')


    if is_trk_id: 
        legend = ROOT.TLegend(0.1, 0.63, 0.85, 0.76)
        le_old = legend.AddEntry(h_old, old_suffix, 'l')
        le_new = legend.AddEntry(h_new, new_suffix, 'l')
        le_old.SetLineWidth(6)
        le_new.SetLineWidth(6)
        le_old.SetLineColor(old_color)
        le_new.SetLineColor(new_color)
        legend.Draw()


c.Draw()
c.SaveAs('{}/grid_seco__{}_vs_{}.png'.format(output_dir, old_suffix_short, new_suffix_short))
c.SaveAs('{}/grid_seco__{}_vs_{}.pdf'.format(output_dir, old_suffix_short, new_suffix_short))

### anglular distribution
visible shape of the detector  
expected, larger difference in the middle  
separate plots for primary/secondary , old/new  
kind of ratio

#### whole

In [ ]:
%%time 

root_style_default()
ROOT.gStyle.SetOptLogy(0)
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetPadTopMargin(0.1)
ROOT.gStyle.SetTitleFontSize(0.07)


c = ROOT.TCanvas('c', 'c', 800, 800)
c.Divide(2,2)

det_id = 29
nbins = 21
h_old = get_histo_2D(t1, 'FV0Hit.mPositionStart.fCoordinates.fX', 'FV0Hit.mPositionStart.fCoordinates.fY', 
                 binning=(BINNINGS['mPositionStart.fCoordinates.fX'], BINNINGS['mPositionStart.fCoordinates.fY']),
#                  select_funcs=[lambda x,y,z: is_in_sector(x,y,z, detector_id_old2new(det_id)), ], 
#                  events_frac=1
                    )
h_new = get_histo_2D(t2, 'FV0Hit.mPositionStart.fCoordinates.fX', 'FV0Hit.mPositionStart.fCoordinates.fY', 
                 binning=(BINNINGS['mPositionStart.fCoordinates.fX'], BINNINGS['mPositionStart.fCoordinates.fY']),
#                  select_funcs=[lambda x,y,z: is_in_sector(x,y,z, det_id), ], 
#                  events_frac=1
                    )


h_old.Scale(1/h_old.GetEntries())
h_new.Scale(1/h_new.GetEntries())

h_diff = h_new.Clone()
h_aver = h_new.Clone()
h_ratio = h_new.Clone()


h_diff.Add(h_old, h_new, -1, 1)
h_aver.Add(h_old, h_new, 0.5, 0.5)
h_ratio.Divide(h_diff, h_aver)

h_old.SetTitle(old_suffix+': PositionStart X vs Y')
h_new.SetTitle(new_suffix+': PositionStart X vs Y')
h_diff.SetTitle('difference: '+new_suffix+' - '+old_suffix)
h_ratio.SetTitle('ratio: difference / average')

c.cd(1)
h_old.Draw('colz')
c.cd(2)
h_new.Draw('colz')
c.cd(3)
h_diff.Draw('colz')
c.cd(4)
h_ratio.Draw('colz')
c.Draw()
c.SaveAs('{}/ang_all__{}_vs_{}.png'.format(output_dir, old_suffix_short, new_suffix_short))
c.SaveAs('{}/ang_all__{}_vs_{}.pdf'.format(output_dir, old_suffix_short, new_suffix_short))

In [ ]:

%%time 

root_style_default()
ROOT.gStyle.SetOptLogy(0)
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetPadTopMargin(0.1)
ROOT.gStyle.SetTitleFontSize(0.07)
# ROOT.gStyle.SetTitleOffset(-100)


c = ROOT.TCanvas('c', 'c', 800, 800)
c.Divide(2,2)

det_id = 29
nbins = 21
h_old = get_histo_2D(t1, 'FV0Hit.mPositionStart.fCoordinates.fX', 'FV0Hit.mPositionStart.fCoordinates.fY', 
                 binning=(BINNINGS['mPositionStart.fCoordinates.fX'], BINNINGS['mPositionStart.fCoordinates.fY']),
#                  select_funcs=[lambda x,y,z: is_in_sector(x,y,z, detector_id_old2new(det_id)), ], 
                 select_funcs=[is_secondary],
#                  events_frac=0.1
                    )
h_new = get_histo_2D(t2, 'FV0Hit.mPositionStart.fCoordinates.fX', 'FV0Hit.mPositionStart.fCoordinates.fY', 
                 binning=(BINNINGS['mPositionStart.fCoordinates.fX'], BINNINGS['mPositionStart.fCoordinates.fY']),
#                  select_funcs=[lambda x,y,z: is_in_sector(x,y,z, det_id), ], 
                 select_funcs=[is_secondary],
#                  events_frac=0.1
                    )


h_old.Scale(1/h_old.GetEntries())
h_new.Scale(1/h_new.GetEntries())

h_diff = h_new.Clone()
h_aver = h_new.Clone()
h_ratio = h_new.Clone()


h_diff.Add(h_old, h_new, -1, 1)
h_aver.Add(h_old, h_new, 0.5, 0.5)
h_ratio.Divide(h_diff, h_aver)

h_old.SetTitle(old_suffix+': PositionStart X vs Y')
h_new.SetTitle(new_suffix+': PositionStart X vs Y')
h_diff.SetTitle('difference: '+new_suffix+' - '+old_suffix)
h_ratio.SetTitle('ratio: difference / average')

c.cd(1)
h_old.Draw('colz')
c.cd(2)
h_new.Draw('colz')
c.cd(3)
h_diff.Draw('colz')
c.cd(4)
h_ratio.Draw('colz')
c.Draw()
c.SaveAs('{}/ang_seco__{}_vs_{}.png'.format(output_dir, old_suffix_short, new_suffix_short))
c.SaveAs('{}/ang_seco__{}_vs_{}.pdf'.format(output_dir, old_suffix_short, new_suffix_short))

In [ ]:

%%time 

root_style_default()
ROOT.gStyle.SetOptLogy(0)
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetPadTopMargin(0.1)
ROOT.gStyle.SetTitleFontSize(0.07)
# ROOT.gStyle.SetTitleOffset(-100)


c = ROOT.TCanvas('c', 'c', 800, 800)
c.Divide(2,2)

det_id = 29
nbins = 21
h_old = get_histo_2D(t1, 'FV0Hit.mPositionStart.fCoordinates.fX', 'FV0Hit.mPositionStart.fCoordinates.fY', 
                 binning=(BINNINGS['mPositionStart.fCoordinates.fX'], BINNINGS['mPositionStart.fCoordinates.fY']),
#                  select_funcs=[lambda x,y,z: is_in_sector(x,y,z, detector_id_old2new(det_id)), ], 
                 select_funcs=[is_primary],
#                  events_frac=0.1
                    )
h_new = get_histo_2D(t2, 'FV0Hit.mPositionStart.fCoordinates.fX', 'FV0Hit.mPositionStart.fCoordinates.fY', 
                 binning=(BINNINGS['mPositionStart.fCoordinates.fX'], BINNINGS['mPositionStart.fCoordinates.fY']),
#                  select_funcs=[lambda x,y,z: is_in_sector(x,y,z, det_id), ], 
                 select_funcs=[is_primary],
#                  events_frac=0.1
                    )


h_old.Scale(1/h_old.GetEntries())
h_new.Scale(1/h_new.GetEntries())

h_diff = h_new.Clone()
h_aver = h_new.Clone()
h_ratio = h_new.Clone()


h_diff.Add(h_old, h_new, -1, 1)
h_aver.Add(h_old, h_new, 0.5, 0.5)
h_ratio.Divide(h_diff, h_aver)

h_old.SetTitle(old_suffix+': PositionStart X vs Y')
h_new.SetTitle(new_suffix+': PositionStart X vs Y')
h_diff.SetTitle('difference: '+new_suffix+' - '+old_suffix)
h_ratio.SetTitle('ratio: difference / average')

c.cd(1)
h_old.Draw('colz')
c.cd(2)
h_new.Draw('colz')
c.cd(3)
h_diff.Draw('colz')
c.cd(4)
h_ratio.Draw('colz')
c.Draw()
c.SaveAs('{}/ang_prim__{}_vs_{}.png'.format(output_dir, old_suffix_short, new_suffix_short))
c.SaveAs('{}/ang_prim__{}_vs_{}.pdf'.format(output_dir, old_suffix_short, new_suffix_short))

#### center zoom

In [ ]:
%%time 

root_style_default()
ROOT.gStyle.SetOptLogy(0)
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetPadTopMargin(0.1)
ROOT.gStyle.SetTitleFontSize(0.07)
ROOT.gStyle.SetPadRightMargin(0.15)
ROOT.gStyle.SetPadLeftMargin(0.05)

c = ROOT.TCanvas('c', 'c', 1200, 800)


n_cols = 3
plot_size = 400
c = ROOT.TCanvas('c', 'c', plot_size*n_cols, plot_size*2)
c.Divide(n_cols,2)

# det_id = 29

for i_pad_shift, binning in zip([0,n_cols], [BINNINGS['mPositionStart.fCoordinates.fX'], (400, -15, 15)]):
    h_old = get_histo_2D(t1, 'FV0Hit.mPositionStart.fCoordinates.fX', 'FV0Hit.mPositionStart.fCoordinates.fY', 
                     binning=(binning, binning),
    #                  select_funcs=[lambda x,y,z: is_in_sector(x,y,z, detector_id_old2new(det_id)), ], 
                     events_frac=1
                        )
    h_new = get_histo_2D(t2, 'FV0Hit.mPositionStart.fCoordinates.fX', 'FV0Hit.mPositionStart.fCoordinates.fY', 
                     binning=(binning, binning),
    #                  select_funcs=[lambda x,y,z: is_in_sector(x,y,z, det_id), ], 
                     events_frac=1
                        )


    h_old.Scale(1/h_old.GetEntries())
    h_new.Scale(1/h_new.GetEntries())

    h_ratio = h_new.Clone()
    h_diff = h_new.Clone()
    h_aver = h_new.Clone()
    h_diff_aver = h_new.Clone()


    h_ratio.Divide(h_new, h_old)
    h_diff.Add(h_old, h_new, -1, 1)
    h_aver.Add(h_old, h_new, 0.5, 0.5)
    h_diff_aver.Divide(h_diff, h_aver)

    h_old.SetTitle(old_suffix+': PositionStart X vs Y')
    h_new.SetTitle(new_suffix+': PositionStart X vs Y')
    h_ratio.SetTitle('ratio: '+new_suffix+' / ' +old_suffix)
    h_diff.SetTitle('difference: '+new_suffix+' - '+old_suffix)
    h_diff_aver.SetTitle('ratio: difference / average')

    c.cd(1+i_pad_shift)
    h_old.DrawCopy('colz')
    c.cd(2+i_pad_shift)
    h_new.DrawCopy('colz')
    c.cd(3+i_pad_shift)
    h_ratio.DrawCopy('colz')

c.Draw()
c.SaveAs('{}/ang_all__{}_vs_{}.png'.format(output_dir, old_suffix_short, new_suffix_short))
c.SaveAs('{}/ang_all__{}_vs_{}.pdf'.format(output_dir, old_suffix_short, new_suffix_short))

# FINAL PLOTS - SANITY CHECKS & OTHERS

In [ ]:
first_color  = ROOT.kBlue
second_color = ROOT.kRed
third_color  = ROOT.kOrange

## particles which passed vs not_passed the detector
their energy?

In [ ]:
first_suffix = 'passed'
second_suffix = 'not passed'
# third_suffix = ''
first_suffix_short = 'pass'
second_suffix_short = 'notPass'
# third_suffix_short = ''
output_dir = 'graphics/tmp'

In [ ]:
root_style_default()
ROOT.gStyle.SetStatX(0.85)


t = t1

c = ROOT.TCanvas('c','c',2400,1200)
c.Divide(5,3)
for i,var in enumerate([leaf.GetName() for leaf in t.GetListOfLeaves() if 'V0Hit.' in leaf.GetName()],1):
    print var,

    is_pdg = 'pdg' in var.lower()
    is_det_id = 'detectorid' in var.lower()
    is_trk_id = 'trackid' in var.lower()

    if is_trk_id:
        h_old = get_histo_n_prim_sec(t, [has_passed], suffix=old_suffix, color=first_color)
        h_new = get_histo_n_prim_sec(t, [has_not_passed], suffix=new_suffix, color=second_color)
    else:
        binning = BINNINGS[var.replace('FV0Hit.', '')]
        if is_pdg:
            f_apply1 = get_pdg_index 
            f_apply2 = get_pdg_index 
        elif is_det_id:
#             f_apply1 = detector_id_new2old
            f_apply1 = lambda x: x
            f_apply2 = lambda x: x
        else:
            f_apply1 = lambda x: x
            f_apply2 = lambda x: x
            
        f_apply = f_apply1
        h_old = get_histo(t, var, f_apply=f_apply, binning=binning, select_funcs=[has_passed])
        h_new = get_histo(t, var, f_apply=f_apply, binning=binning, select_funcs=[has_not_passed])
    
    h_sum  = ROOT.TH1D()
    h_old.Copy(h_sum)
    h_sum.Add(h_new)
    if is_trk_id:
        h_sum.SetTitle('n prim. & seco.')
        h_sum.SetLabelSize(0.07)

    if is_pdg:  
        for i_bin, code in enumerate(PDG_CODES_SORTED,1):
            name = get_pdg_name_latex(code)
            h_sum.GetXaxis().SetBinLabel(i_bin, name)
        h_sum.SetLabelSize(0.05)

    h_sum.SetLineColor(ROOT.kWhite)
    h_old.SetLineColor(first_color)
    h_new.SetLineColor(second_color)
    h_sum.Scale(1/h_sum.GetEntries())
    h_old.Scale(1/h_old.GetEntries())
    h_new.Scale(1/h_new.GetEntries())
    
    c.cd(i)
    if is_trk_id:
        ROOT.gPad.SetLogy(0)
#         h_sum.SetMinimum(0)
    else:
        ROOT.gPad.SetLogy(1)
    h_sum.DrawCopy('hist')
    h_old.DrawCopy('hist same')
    h_new.DrawCopy('hist same')

    if is_trk_id: 
        legend = ROOT.TLegend(0.1, 0.63, 0.45, 0.76)
        le_old = legend.AddEntry(h_old, first_suffix, 'l')
        le_new = legend.AddEntry(h_new, second_suffix, 'l')
        le_old.SetLineWidth(6)
        le_new.SetLineWidth(6)
        le_old.SetLineColor(first_color)
        le_new.SetLineColor(second_color)
        legend.Draw()


c.Draw()
c.SaveAs('{}/grid_{}__{}_vs_{}.png'.format(output_dir, old_suffix_short, first_suffix_short, second_suffix_short))
c.SaveAs('{}/grid_{}__{}_vs_{}.pdf'.format(output_dir, old_suffix_short, first_suffix_short, second_suffix_short))

In [ ]:
root_style_default()
ROOT.gStyle.SetStatX(0.85)


t = t2

c = ROOT.TCanvas('c','c',2400,1200)
c.Divide(5,3)
for i,var in enumerate([leaf.GetName() for leaf in t.GetListOfLeaves() if 'FV0Hit.' in leaf.GetName()],1):
    print var,

    is_pdg = 'pdg' in var.lower()
    is_det_id = 'detectorid' in var.lower()
    is_trk_id = 'trackid' in var.lower()

    if is_trk_id:
        h_old = get_histo_n_prim_sec(t, [has_passed], suffix=old_suffix, color=first_color)
        h_new = get_histo_n_prim_sec(t, [has_not_passed], suffix=new_suffix, color=second_color)
    else:
        binning = BINNINGS[var.replace('FV0Hit.', '')]
        if is_pdg:
            f_apply1 = get_pdg_index 
            f_apply2 = get_pdg_index 
        elif is_det_id:
#             f_apply1 = detector_id_new2old
            f_apply1 = lambda x: x
            f_apply2 = lambda x: x
        else:
            f_apply1 = lambda x: x
            f_apply2 = lambda x: x
        
        f_apply = f_apply2
        h_old = get_histo(t, var, f_apply=f_apply, binning=binning, select_funcs=[has_passed])
        h_new = get_histo(t, var, f_apply=f_apply, binning=binning, select_funcs=[has_not_passed])
    
    h_sum  = ROOT.TH1D()
    h_old.Copy(h_sum)
    h_sum.Add(h_new)
    if is_trk_id:
        h_sum.SetTitle('n prim. & seco.')
        h_sum.SetLabelSize(0.07)

    if is_pdg:  
        for i_bin, code in enumerate(PDG_CODES_SORTED,1):
            name = get_pdg_name_latex(code)
            h_sum.GetXaxis().SetBinLabel(i_bin, name)
        h_sum.SetLabelSize(0.05)
    h_sum.SetLineColor(ROOT.kWhite)
    h_old.SetLineColor(first_color)
    h_new.SetLineColor(second_color)
    h_sum.Scale(1/h_sum.GetEntries())
    h_old.Scale(1/h_old.GetEntries())
    h_new.Scale(1/h_new.GetEntries())
    
    c.cd(i)
    if is_trk_id:
        ROOT.gPad.SetLogy(0)
#         h_sum.SetMinimum(0)
    else:
        ROOT.gPad.SetLogy(1)
    h_sum.DrawCopy('hist')
    h_old.DrawCopy('hist same')
    h_new.DrawCopy('hist same')


    if is_trk_id: 
        legend = ROOT.TLegend(0.1, 0.63, 0.45, 0.76)
        le_old = legend.AddEntry(h_old, first_suffix, 'l')
        le_new = legend.AddEntry(h_new, second_suffix, 'l')
        le_old.SetLineWidth(6)
        le_new.SetLineWidth(6)
        le_old.SetLineColor(first_color)
        le_new.SetLineColor(second_color)
        legend.Draw()


c.Draw()
c.SaveAs('{}/grid_{}__{}_vs_{}.png'.format(output_dir, new_suffix_short, first_suffix_short, second_suffix_short))
c.SaveAs('{}/grid_{}__{}_vs_{}.pdf'.format(output_dir, new_suffix_short, first_suffix_short, second_suffix_short))

## e+/e- vs gamma vs others
their Eloss, Estart?

In [ ]:
first_suffix = 'other'
second_suffix = 'e+/e-'
third_suffix = 'gamma'
first_suffix_short = 'other'
second_suffix_short = 'e'
third_suffix_short = 'gamma'
output_dir = 'graphics/tmp'

In [ ]:

root_style_default()
ROOT.gStyle.SetStatX(0.85)


t = t1

c = ROOT.TCanvas('c','c',2400,1200)
c.Divide(5,3)
for i,var in enumerate([leaf.GetName() for leaf in t.GetListOfLeaves() if 'V0Hit.' in leaf.GetName()],1):
    print var,

    is_pdg = 'pdg' in var.lower()
    is_det_id = 'detectorid' in var.lower()
    is_trk_id = 'trackid' in var.lower()

    if is_trk_id:
        h_ele = get_histo_n_prim_sec(t, [is_electron], suffix=old_suffix, color=first_color)
        h_other = get_histo_n_prim_sec(t, [is_not_electron, is_not_gamma], suffix=new_suffix, color=second_color)
        h_gamma = get_histo_n_prim_sec(t, [is_gamma], suffix=new_suffix, color=third_color)
    else:
        binning = BINNINGS[var.replace('FV0Hit.', '')]
        if is_pdg:
            f_apply1 = get_pdg_index 
            f_apply2 = get_pdg_index 
        elif is_det_id:
#             f_apply1 = detector_id_new2old
            f_apply1 = lambda x: x
            f_apply2 = lambda x: x
        else:
            f_apply1 = lambda x: x
            f_apply2 = lambda x: x
            
        f_apply = f_apply1
        h_ele = get_histo(t, var, f_apply=f_apply, binning=binning, select_funcs=[is_electron])
        h_other = get_histo(t, var, f_apply=f_apply, binning=binning, select_funcs=[is_not_electron, is_not_gamma])
        h_gamma = get_histo(t, var, f_apply=f_apply, binning=binning, select_funcs=[is_gamma])
    
    h_sum  = ROOT.TH1D()
    h_other.Copy(h_sum)
    h_sum.Add(h_ele)
    h_sum.Add(h_gamma)
    if is_trk_id:
        h_sum.SetTitle('n prim. & seco.')
        h_sum.SetLabelSize(0.07)

    if is_pdg:  
        for i_bin, code in enumerate(PDG_CODES_SORTED,1):
            name = get_pdg_name_latex(code)
            h_sum.GetXaxis().SetBinLabel(i_bin, name)
        h_sum.SetLabelSize(0.07)

    h_sum.SetLineColor(ROOT.kWhite)
    h_other.SetLineColor(first_color)
    h_ele.SetLineColor(second_color)
    h_gamma.SetLineColor(third_color)
    h_sum.Scale(1/h_sum.GetEntries())
    h_other.Scale(1/h_other.GetEntries())
    h_ele.Scale(1/h_ele.GetEntries())
    h_gamma.Scale(1/h_gamma.GetEntries())
    
    c.cd(i)
    if is_trk_id:
        ROOT.gPad.SetLogy(0)
        h_sum.SetMaximum(1)
#         h_sum.SetMinimum(0)
    else:
        ROOT.gPad.SetLogy(1)
    h_sum.DrawCopy('hist')
    h_other.DrawCopy('hist same')
    h_ele.DrawCopy('hist same')
    h_gamma.DrawCopy('hist same')


    if is_trk_id: 
        legend = ROOT.TLegend(0.1, 0.58, 0.45, 0.76)
        le_ele = legend.AddEntry(h_ele, second_suffix, 'l')
        le_gamma = legend.AddEntry(h_gamma, third_suffix, 'l')
        le_other = legend.AddEntry(h_other, first_suffix, 'l')
        le_other.SetLineWidth(6)
        le_ele.SetLineWidth(6)
        le_gamma.SetLineWidth(6)
        le_other.SetLineColor(first_color)
        le_ele.SetLineColor(second_color)
        le_gamma.SetLineColor(third_color)
        legend.Draw()


c.Draw()
c.SaveAs('{}/grid_{}__{}_vs_{}_vs_{}.png'.format(output_dir, old_suffix_short, first_suffix_short, second_suffix_short, third_suffix_short))
c.SaveAs('{}/grid_{}__{}_vs_{}_vs_{}.pdf'.format(output_dir, old_suffix_short, first_suffix_short, second_suffix_short, third_suffix_short))

In [ ]:
root_style_default()
ROOT.gStyle.SetStatX(0.85)


t = t2

c = ROOT.TCanvas('c','c',2400,1200)
c.Divide(5,3)
for i,var in enumerate([leaf.GetName() for leaf in t.GetListOfLeaves() if 'FV0Hit.' in leaf.GetName()],1):
    print var,

    is_pdg = 'pdg' in var.lower()
    is_det_id = 'detectorid' in var.lower()
    is_trk_id = 'trackid' in var.lower()

    if is_trk_id:
        h_ele = get_histo_n_prim_sec(t, [is_electron], suffix=old_suffix, color=first_color)
        h_other = get_histo_n_prim_sec(t, [is_not_electron, is_not_gamma], suffix=new_suffix, color=second_color)
        h_gamma = get_histo_n_prim_sec(t, [is_gamma], suffix=new_suffix, color=third_color)
    else:
        binning = BINNINGS[var.replace('FV0Hit.', '')]
        if is_pdg:
            f_apply1 = get_pdg_index 
            f_apply2 = get_pdg_index 
        elif is_det_id:
#             f_apply1 = detector_id_new2old
            f_apply1 = lambda x: x
            f_apply2 = lambda x: x
        else:
            f_apply1 = lambda x: x
            f_apply2 = lambda x: x
            
        f_apply = f_apply1
        h_ele = get_histo(t, var, f_apply=f_apply, binning=binning, select_funcs=[is_electron])
        h_other = get_histo(t, var, f_apply=f_apply, binning=binning, select_funcs=[is_not_electron, is_not_gamma])
        h_gamma = get_histo(t, var, f_apply=f_apply, binning=binning, select_funcs=[is_gamma])
    
    h_sum  = ROOT.TH1D()
    h_other.Copy(h_sum)
    h_sum.Add(h_ele)
    h_sum.Add(h_gamma)
    if is_trk_id:
        h_sum.SetTitle('n prim. & seco.')
        h_sum.SetLabelSize(0.07)

    if is_pdg:  
        for i_bin, code in enumerate(PDG_CODES_SORTED,1):
            name = get_pdg_name_latex(code)
            h_sum.GetXaxis().SetBinLabel(i_bin, name)
        h_sum.SetLabelSize(0.07)

    h_sum.SetLineColor(ROOT.kWhite)
    h_other.SetLineColor(first_color)
    h_ele.SetLineColor(second_color)
    h_gamma.SetLineColor(third_color)
    h_sum.Scale(1/h_sum.GetEntries())
    h_other.Scale(1/h_other.GetEntries())
    h_ele.Scale(1/h_ele.GetEntries())
    h_gamma.Scale(1/h_gamma.GetEntries())
    
    c.cd(i)
    if is_trk_id:
        ROOT.gPad.SetLogy(0)
        h_sum.SetMaximum(1)
#         h_sum.SetMinimum(0)
    else:
        ROOT.gPad.SetLogy(1)
    h_sum.DrawCopy('hist')
    h_other.DrawCopy('hist same')
    h_ele.DrawCopy('hist same')
    h_gamma.DrawCopy('hist same')


    if is_trk_id: 
        legend = ROOT.TLegend(0.1, 0.58, 0.45, 0.76)
        le_ele = legend.AddEntry(h_ele, second_suffix, 'l')
        le_gamma = legend.AddEntry(h_gamma, third_suffix, 'l')
        le_other = legend.AddEntry(h_other, first_suffix, 'l')
        le_other.SetLineWidth(6)
        le_ele.SetLineWidth(6)
        le_gamma.SetLineWidth(6)
        le_other.SetLineColor(first_color)
        le_ele.SetLineColor(second_color)
        le_gamma.SetLineColor(third_color)
        legend.Draw()


c.Draw()
c.SaveAs('{}/grid_{}__{}_vs_{}_vs_{}.png'.format(output_dir, new_suffix_short, first_suffix_short, second_suffix_short, third_suffix_short))
c.SaveAs('{}/grid_{}__{}_vs_{}_vs_{}.pdf'.format(output_dir, new_suffix_short, first_suffix_short, second_suffix_short, third_suffix_short))

# TODO:

- save orig vs newSensOnly plots in that form, then compare only newSensOnly with newFull -- DONE
- second angular distr. with center pads - higher granurality -- DONE
- ratio o angular distr. -- DONE
- \# particles stopped at the front wall, \# particles passed, difference after adding fibres and plastic -- DONE